<a href="https://colab.research.google.com/github/demekeendalie/Fine-tuning-bert-model-for-amharic-news-classification/blob/main/Amharic_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
#!pip install -q openpyxl

In [2]:
import pandas as pd
import numpy as np
import os

In [131]:
df = pd.read_csv("labeled_data_final.csv")

In [132]:
# Display the first 5 rows
df.head()

,text,label
0,"የሚከራይ ቤት አከባቢ ዋጋ: 120,000 በወር የቤት አይነት: መኝታ: 3...",real_estate
1,"የሚከራይ ቤት 72 አከባቢ ዋጋ: 37,000 በወር የቤት አይነት: +2 መ...",real_estate
2,"የሚከራይ ቤት አከባቢ ዋጋ: 35,000 በወር የቤት አይነት: መኝታ: 3 ...",real_estate
3,"የሚከራይ ቤት አከባቢ ዋጋ: 28,000 በወር የቤት አይነት: 40/60 መ...",real_estate
4,"የሚከራይ ቤት አከባቢ ዋጋ: 45,000 በወር የቤት አይነት: መኝታ: 2 ...",real_estate


In [133]:
df.shape

(17189, 2)

In [134]:
# Different category
df["label"].unique()

array(['real_estate', 'retail', 'sport', 'economy', 'politics',
       'accident', 'agriculture', 'health', 'education', ' Education',
       ' Financial services', ' Charity', ' CPG & consumer products',
       ' Healthcare & Pharma', ' Travel', ' Telecom', ' Media',
       ' Healthcare & pharma', ' Entertainment', ' Retail',
       ' Computing products & consumer electronics', ' Gambling',
       ' Technology', ' Automotive', ' Real Estate',
       'advertisements - Computing products & consumer electronics',
       ' Event & Entertainment', ' Financial Services',
       ' Event Organizer', 'advertisement - Event Organizer',
       'whether the data is an advertisements - Travel',
       'whether the data is an advertisements - Retail', ' Services',
       ' Computing Products & Consumer Electronics',
       'advertisement - Real Estate', 'advertisements - Retail',
       'advertisement - Travel', 'advertisement - Entertainment',
       'advertisement - Healthcare & Pharma', 'adverti

### changing label type to numeric

In [135]:

label_value = pd.Categorical(df["label"]).codes 
label_value = pd.DataFrame(label_value)
df["label_value"] = label_value
print(df)

                                                    text        label  \
0      የሚከራይ ቤት አከባቢ ዋጋ: 120,000 በወር የቤት አይነት: መኝታ: 3...  real_estate   
1      የሚከራይ ቤት 72 አከባቢ ዋጋ: 37,000 በወር የቤት አይነት: +2 መ...  real_estate   
2      የሚከራይ ቤት አከባቢ ዋጋ: 35,000 በወር የቤት አይነት: መኝታ: 3 ...  real_estate   
3      የሚከራይ ቤት አከባቢ ዋጋ: 28,000 በወር የቤት አይነት: 40/60 መ...  real_estate   
4      የሚከራይ ቤት አከባቢ ዋጋ: 45,000 በወር የቤት አይነት: መኝታ: 2 ...  real_estate   
...                                                  ...          ...   
17184  ከች ብለናል ከሳፋሪኮም! ዘወትር ቅዳሜ ከሰዓት በEBS TV ላይ! ሰሞነኛ...      Telecom   
17185  አዲሱን ያልተገደበ የኢንተርኔት ጥቅላችንን እንደልብ እየተጠቀምን የእረፍት...      Telecom   
17186  ከዚህ ሁሉ የተረክ በM PESA ሽልማት አይን ጨፍናችሁ ብትዘግኑ አንድ አ...      Telecom   
17187  መልካም ሰኞ! ያልተገደበ የኢንተርኔት አገልግሎታችንን በM PESA App ...      Telecom   
17188  ኢንተርኔት እንደልብ መሆኑን ሰምታችኋል?! በ999 ብር ብቻ ያልተገደበ ወ...      Telecom   

       label_value  
0               62  
1               62  
2               62  
3               62  
4               62

In [136]:
df["label_value"].nunique()

70

#### Make a copy before deleting the category feature

In [137]:
data = df.copy()
data.head(3)

,text,label,label_value
0,"የሚከራይ ቤት አከባቢ ዋጋ: 120,000 በወር የቤት አይነት: መኝታ: 3...",real_estate,62
1,"የሚከራይ ቤት 72 አከባቢ ዋጋ: 37,000 በወር የቤት አይነት: +2 መ...",real_estate,62
2,"የሚከራይ ቤት አከባቢ ዋጋ: 35,000 በወር የቤት አይነት: መኝታ: 3 ...",real_estate,62


In [138]:
# remove label category

del data["label"]
print(data)

                                                    text  label_value
0      የሚከራይ ቤት አከባቢ ዋጋ: 120,000 በወር የቤት አይነት: መኝታ: 3...           62
1      የሚከራይ ቤት 72 አከባቢ ዋጋ: 37,000 በወር የቤት አይነት: +2 መ...           62
2      የሚከራይ ቤት አከባቢ ዋጋ: 35,000 በወር የቤት አይነት: መኝታ: 3 ...           62
3      የሚከራይ ቤት አከባቢ ዋጋ: 28,000 በወር የቤት አይነት: 40/60 መ...           62
4      የሚከራይ ቤት አከባቢ ዋጋ: 45,000 በወር የቤት አይነት: መኝታ: 2 ...           62
...                                                  ...          ...
17184  ከች ብለናል ከሳፋሪኮም! ዘወትር ቅዳሜ ከሰዓት በEBS TV ላይ! ሰሞነኛ...           33
17185  አዲሱን ያልተገደበ የኢንተርኔት ጥቅላችንን እንደልብ እየተጠቀምን የእረፍት...           33
17186  ከዚህ ሁሉ የተረክ በM PESA ሽልማት አይን ጨፍናችሁ ብትዘግኑ አንድ አ...           33
17187  መልካም ሰኞ! ያልተገደበ የኢንተርኔት አገልግሎታችንን በM PESA App ...           33
17188  ኢንተርኔት እንደልብ መሆኑን ሰምታችኋል?! በ999 ብር ብቻ ያልተገደበ ወ...           33

[17189 rows x 2 columns]


In [139]:
# mixing each category and their labels. 
data = data.sample(frac=1)
data.head()

,text,label_value
635,"የሚከራይ ቤት 72 አከባቢ ዋጋ: 9,000 በወር የቤት አይነት: መኝታ: ...",62
3547,Fashion SHOE Size: 43 44 Made in Vietnam Price...,63
12579,Fashion SHOE Size: 43 44 Made in Vietnam Price...,63
5170,Adidas Duramo 9 Size: 40 41 42 43 Made in Viet...,63
12172,"የሚከራይ ቤት 49 ዋጋ: 25,000 በወር የቤት አይነት: መኝታ: 2 መታ...",62


In [140]:
# creating word_len feature where we count the length of each text
data["text_len"] = data["text"].str.split().str.len()
# mean of articles length
data["text_len"].mean() 

74.27290709174473

In [141]:
data.head(3)

,text,label_value,text_len
635,"የሚከራይ ቤት 72 አከባቢ ዋጋ: 9,000 በወር የቤት አይነት: መኝታ: ...",62,25
3547,Fashion SHOE Size: 43 44 Made in Vietnam Price...,63,22
12579,Fashion SHOE Size: 43 44 Made in Vietnam Price...,63,22


In [142]:
#number of news under each categories
#accident=dataframe[dataframe['category']=='accident']
#agir=dataframe[dataframe['category']=='agriculture']
#health=dataframe[dataframe['category']=='health']
#education=dataframe[dataframe['category']=='education']
#economy=dataframe[dataframe['category']=='economy']
#politics=dataframe[dataframe['category']=='politics']
#sport=dataframe[dataframe['category']=='sport']
#print(accident.shape)
#print(agir.shape)
#print(health.shape)
#print(education.shape)
#print(economy.shape)
#print(politics.shape)
#print(sport.shape)

In [143]:
#normalizarion
import re
#method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalization(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ
    return rep48

#### clean the text

In [144]:
# apply normalizaton to text
data["text"] = data["text"].apply(lambda x: normalization(x))

In [145]:
# remove special characters
data["text"] = data["text"].str.replace("[^\w\s]", "", regex=True)

In [146]:
# remove digits from the dataset
data["text"] = data["text"].str.replace("\d+", "", regex= True)

In [147]:
# display our clean data.
print(data)

                                                    text  label_value  \
635    የሚከራይ ቤት  አከባቢ ዋጋ  በወር የቤት አይነት መኝታ  መታጠቢያ  ደረ...           62   
3547   Fashion SHOE Size   Made in Vietnam Price Br  ...           63   
12579  Fashion SHOE Size   Made in Vietnam Price Br  ...           63   
5170   Adidas Duramo  Size     Made in Vietnam Price ...           63   
12172  የሚከራይ ቤት  ዋጋ  በወር የቤት አይነት መኝታ  መታጠቢያ  ደረጃ th ...           62   
...                                                  ...          ...   
11284  የሚከራይ ቤት አከባቢ ዋጋ  በወር የቤት አይነት መኝታ  መታጠቢያ  ደረጃ...           62   
11964  Bapesta Sneaker Size     Made in Vietnam Price...           63   
5390   Dental White ጥርስ ማንጫ    ፈገግታወን ልንመልስ ሀጫ በረዶ የመ...           63   
860    LV TRAINER SNEAKER Size     Made in Italy Pric...           63   
15795  ADVERTISEMENT ለኢንጅነሮች እና ኢንጂነሪንግ ተማሪዎች  የኮንስትራ...           60   

       text_len  
635          25  
3547         22  
12579        22  
5170         25  
12172        33  
...         ...

#### Train and Test datasets

In [148]:
!pip install -q scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [149]:
from sklearn.model_selection import train_test_split

train_evaluation_dataset, test_dataset = train_test_split(data, test_size=0.20, random_state=42) 
train_dataset, evaluation_dataset = train_test_split(train_evaluation_dataset, test_size=0.10, random_state=42)
print(f"Tranining dataset shape: {train_dataset.shape}")
print(f"Testing dataset shape: {test_dataset.shape}")
print(f"Evaluation dataset shape: {evaluation_dataset.shape}")


Tranining dataset shape: (12375, 3)
Testing dataset shape: (3438, 3)
Evaluation dataset shape: (1376, 3)


In [150]:
# Another way to split your train and test datasets

# msk = np.random.rand(len(data))< 0.8
# train_dataset = data[msk]
# test_dataset = data[~msk]

#### Prepare a datasets from pandas dataframe

In [151]:
!pip install -q datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [152]:
from datasets import Dataset

In [153]:
# converting the pandas format of our data into HuggingFace Dataset format

train_dataset = Dataset.from_pandas(train_dataset)
train_dataset

Dataset({
    features: ['text', 'label_value', 'text_len', '__index_level_0__'],
    num_rows: 12375
})

In [154]:
test_dataset = Dataset.from_pandas(test_dataset)
test_dataset

Dataset({
    features: ['text', 'label_value', 'text_len', '__index_level_0__'],
    num_rows: 3438
})

In [155]:
evaluation_dataset = Dataset.from_pandas(evaluation_dataset)
evaluation_dataset

Dataset({
    features: ['text', 'label_value', 'text_len', '__index_level_0__'],
    num_rows: 1376
})

In [157]:
print(test_dataset["label_value"])

[33, 63, 65, 63, 36, 63, 63, 63, 48, 62, 63, 63, 63, 63, 63, 63, 36, 63, 54, 62, 65, 63, 63, 63, 33, 63, 62, 33, 63, 45, 63, 63, 63, 63, 63, 63, 62, 63, 62, 63, 62, 63, 63, 63, 36, 63, 63, 63, 63, 63, 62, 63, 63, 63, 63, 63, 63, 63, 54, 63, 63, 58, 63, 63, 63, 63, 63, 63, 62, 63, 45, 62, 54, 63, 63, 63, 63, 63, 63, 63, 56, 63, 63, 63, 63, 63, 63, 63, 30, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 61, 63, 45, 63, 58, 63, 63, 63, 62, 63, 63, 63, 63, 61, 63, 63, 62, 63, 62, 63, 63, 56, 63, 63, 63, 63, 63, 62, 63, 63, 63, 51, 63, 58, 62, 63, 63, 63, 63, 57, 63, 63, 63, 63, 63, 62, 63, 63, 63, 63, 63, 63, 63, 60, 63, 59, 53, 62, 63, 63, 63, 62, 63, 63, 63, 63, 63, 63, 63, 63, 62, 65, 54, 63, 53, 63, 63, 63, 63, 63, 33, 63, 63, 63, 63, 62, 63, 63, 63, 63, 63, 63, 63, 33, 62, 62, 63, 61, 45, 63, 63, 63, 63, 63, 62, 63, 63, 63, 63, 63, 63, 65, 63, 62, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 63, 57, 62, 63, 63, 63, 63, 63, 63, 61, 63, 63, 61, 62, 63, 57, 63, 65, 63, 63, 63,

In [158]:
# Remove unnecessary column

train_dataset = train_dataset.remove_columns("__index_level_0__")
test_dataset = test_dataset.remove_columns("__index_level_0__")
evaluation_dataset = evaluation_dataset.remove_columns("__index_level_0__")

In [159]:
train_dataset

Dataset({
    features: ['text', 'label_value', 'text_len'],
    num_rows: 12375
})

In [160]:
# combine the train_dataset, test_dataset, and evaluation_dataset into one dataset

import datasets

main_dataset = datasets.DatasetDict(
    {'train': train_dataset,
     'test': test_dataset,
     'evaluation': evaluation_dataset
    }
)

In [161]:
# Display the main_dataset, which is the Dataset dictionary of train, test, and evaluation datasets

main_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label_value', 'text_len'],
        num_rows: 12375
    })
    test: Dataset({
        features: ['text', 'label_value', 'text_len'],
        num_rows: 3438
    })
    evaluation: Dataset({
        features: ['text', 'label_value', 'text_len'],
        num_rows: 1376
    })
})

In [162]:
# training, testing, and evaluation dataset size (number of rows)

train_data_size = main_dataset['train'].num_rows
test_data_size = main_dataset['test'].num_rows
evaluation_data_size = main_dataset['evaluation'].num_rows

In [163]:
train_data_size, test_data_size, evaluation_data_size

(12375, 3438, 1376)

In [164]:
train_data_size + test_data_size + evaluation_data_size

17189

#### Tokenization

In [36]:
from transformers import AutoTokenizer

In [168]:
model_base_name = "Davlan/bert-base-multilingual-cased-finetuned-amharic"

model_llama2_name = "meta-llama/Llama-2-7b-hf"
model_garri_name = "iocuydi/llama-2-amharic-3784m" 

#### Loading a tokenizer from the pretrained model

In [169]:
#loading a tokenizer from the pretrained model

tokenizer = AutoTokenizer.from_pretrained(model_base_name)

tokenizer_llama2 = AutoTokenizer.from_pretrained(model_llama2_name)
tokenizer_garri = AutoTokenizer.from_pretrained(model_garri_name)

In [170]:
# have a tokenizer function that use the tokenizer

def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation = True)
    
def tokenize_function_llama2(data):
    return tokenizer_llama2(data["text"], padding="max_length", truncation = True)

def tokenize_function_garri(data):
    return tokenizer_garri(data["text"], padding="max_length", truncation = True)
    

In [171]:
# Tokenize all the data using ther mapping functionality

tokenized_dataset = main_dataset.map(tokenize_function)

tokenized_dataset_llama2 = main_dataset.map(tokenize_function_llama2)
tokenized_dataset_garri = main_dataset.map(tokenize_function_garri)

Map:   0%|          | 0/12375 [00:00<?, ? examples/s]

Map:   0%|          | 0/3438 [00:00<?, ? examples/s]

Map:   0%|          | 0/1376 [00:00<?, ? examples/s]

Map:   0%|          | 0/12375 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

Map:   0%|          | 0/1376 [00:00<?, ? examples/s]

Map:   0%|          | 0/12375 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3438 [00:00<?, ? examples/s]

Map:   0%|          | 0/1376 [00:00<?, ? examples/s]

In [172]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12375
    })
    test: Dataset({
        features: ['text', 'label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3438
    })
    evaluation: Dataset({
        features: ['text', 'label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1376
    })
})

In [173]:
# remove the column text as it is no longer needed

tokenized_dataset = tokenized_dataset.remove_columns(["text"])

tokenized_dataset_llama2 = tokenized_dataset_llama2.remove_columns(["text"])
tokenized_dataset_garri = tokenized_dataset_garri.remove_columns(["text"])

In [174]:
tokenized_dataset_llama2 

DatasetDict({
    train: Dataset({
        features: ['label_value', 'text_len', 'input_ids', 'attention_mask'],
        num_rows: 12375
    })
    test: Dataset({
        features: ['label_value', 'text_len', 'input_ids', 'attention_mask'],
        num_rows: 3438
    })
    evaluation: Dataset({
        features: ['label_value', 'text_len', 'input_ids', 'attention_mask'],
        num_rows: 1376
    })
})

In [175]:
# import torch
import torch
# import DataLoader from torch
from torch.utils.data import DataLoader
# import an Adam optimizer from torch
from torch.optim import AdamW
# import tqdm for a bar progress
from tqdm.auto import tqdm

In [176]:
# changing the format of the tokenized dataset to torch

tokenized_dataset.set_format("torch")

tokenized_dataset_llama2.set_format("torch")
tokenized_dataset_garri.set_format("torch")

In [177]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12375
    })
    test: Dataset({
        features: ['label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3438
    })
    evaluation: Dataset({
        features: ['label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1376
    })
})

#### Shuffeling and selecting the neededd size of dataset for training and evaluating the model

In [178]:

# with model_base dataset
train_tokenized_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(train_data_size))
test_tokenized_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(test_data_size))
evaluation_tokenized_dataset = tokenized_dataset["evaluation"].shuffle(seed=42).select(range(evaluation_data_size)) 


In [179]:
# with model_llama2 dataset
train_tokenized_dataset_llama2 = tokenized_dataset_llama2["train"].shuffle(seed=42).select(range(train_data_size))
test_tokenized_dataset_llama2 = tokenized_dataset_llama2["test"].shuffle(seed=42).select(range(test_data_size))
evaluation_tokenized_dataset_llama2 = tokenized_dataset_llama2["evaluation"].shuffle(seed=42).select(range(evaluation_data_size)) 


In [180]:

# with model_Garri dataset
train_tokenized_dataset_garri = tokenized_dataset_garri["train"].shuffle(seed=42).select(range(train_data_size))
test_tokenized_dataset_garri = tokenized_dataset_garri["test"].shuffle(seed=42).select(range(test_data_size))
evaluation_tokenized_dataset_garri = tokenized_dataset_garri["evaluation"].shuffle(seed=42).select(range(evaluation_data_size)) 


In [181]:
train_tokenized_dataset

Dataset({
    features: ['label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12375
})

In [182]:
test_tokenized_dataset

Dataset({
    features: ['label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3438
})

#### Load with DataLoader

In [183]:
# Load with DataLoader
train_dataloader = DataLoader(train_tokenized_dataset, shuffle=True, batch_size=4)
test_dataloader = DataLoader(test_tokenized_dataset, shuffle= True, batch_size=4)
evaluation_dataloader = DataLoader(evaluation_tokenized_dataset, shuffle= True, batch_size=4)

In [184]:
# llama2a
train_dataloader_llama2 = DataLoader(train_tokenized_dataset_llama2, shuffle=True, batch_size=4)
test_dataloader_llama2 = DataLoader(test_tokenized_dataset_llama2, shuffle= True, batch_size=4)
evaluation_dataloader_llama2 = DataLoader(evaluation_tokenized_dataset_llama2, shuffle= True, batch_size=4)


In [185]:

# Garri
train_dataloader_garri = DataLoader(train_tokenized_dataset_garri, shuffle=True, batch_size=4)
test_dataloader_garri = DataLoader(test_tokenized_dataset_garri, shuffle= True, batch_size=4)
evaluation_dataloader_garri = DataLoader(evaluation_tokenized_dataset_garri, shuffle= True, batch_size=4)

#### Load AutoModelForSequenceClassification, AutoModelForCausalLMa from the pretrained model from transformers

In [186]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForCausalLM
from huggingface_hub import notebook_login

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

In [70]:
notebook_login()

In [187]:
# from transformers import AutoModel

# # Load the model
# model = AutoModel.from_pretrained('iocuydi/llama-2-amharic-3784m')

# # Generate the config.json file
# model.config.to_json_file("config.json")


#### Function  to download LLaMA 2 model and its tokenizer. It requires a bitsandbytes configuration

In [188]:
def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

#method from the Hugging Face Transformers library to load a pre-trained language model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [189]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments

def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config
    

In [190]:
bnb_config = create_bnb_config()

In [66]:
# Bert model
# model_b, tokenizer_b = load_model(model_base_name, bnb_config)

In [64]:
# Llama2 model
model_l, tokenizer_l = load_model(model_llama2_name, bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/miniconda/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:711: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [75]:
# model_l

In [68]:
# Garri model
# model_g, tokenizer_g = load_model(model_garri_name, bnb_config)

In [191]:
model_b = AutoModelForSequenceClassification.from_pretrained("Davlan/bert-base-multilingual-cased-finetuned-amharic", num_labels=7)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [192]:
# create an instance of AutoModelForSequenceClassification for each LLM

model = AutoModelForSequenceClassification.from_pretrained(model_base_name, num_labels=7)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Davlan/bert-base-multilingual-cased-finetuned-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [194]:
# model

In [195]:
model_llama2 = AutoModelForSequenceClassification.from_pretrained(model_llama2_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [196]:
# model_llama2

In [197]:
# model_garri = AutoModelForSequenceClassification.from_pretrained(model_garri_name, cache_dir="/home/rodolphe_segbedji/.cache")

In [198]:
device = torch.device("cpu")
if torch.cuda.is_available():
    print("Training on GPU")
    device = torch.device("cuda:0")
print(device)

Training on GPU
cuda:0


In [199]:
import numpy as np
from datasets import load_metric
#metric = load_metric("f1", "accuracy")

In [200]:

#def compute_metrics(eval_pred):
 #   logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
 #   return metric.compute(predictions=predictions, references=labels, average="weighted")

In [202]:
# ! pip install -U accelerate
# ! pip install -U transformers

In [203]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [204]:
import accelerate
accelerate.__version__

'0.26.1'

In [205]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy

In [206]:
def custom_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]

    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [207]:
# label_names=['text']
train_tokenized_dataset

Dataset({
    features: ['label_value', 'text_len', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 12375
})

In [220]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy='epoch',
   save_strategy='epoch',
   num_train_epochs=10,
   # total number of training epochs
   learning_rate=1e-5,
   per_device_train_batch_size=64,  # batch size per device during training
   per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=1000,                # number of warmup steps for learning rate
    load_best_model_at_end= True,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=20,

   )

In [221]:
trainer = Trainer(
    model=model_b,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=evaluation_tokenized_dataset,
    compute_metrics=custom_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10, )],
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 176.00 MiB. GPU 0 has a total capacty of 22.02 GiB of which 56.25 MiB is free. Including non-PyTorch memory, this process has 8.91 GiB memory in use. Process 3953 has 6.62 GiB memory in use. Process 8094 has 6.44 GiB memory in use. Of the allocated memory 7.77 GiB is allocated by PyTorch, and 109.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

# with model_base dataset
train_tokenized_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(train_data_size))
test_tokenized_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(test_data_size))
evaluation_tokenized_dataset = tokenized_dataset["evaluation"].shuffle(seed=42).select(range(evaluation_data_size)) 


In [216]:
import torch
torch.cuda.empty_cache()


In [217]:
trainer.train()

AssertionError: EarlyStoppingCallback requires load_best_model_at_end = True

In [59]:
trainer.evaluate(test_dataloader)

{'eval_loss': 0.5854920148849487,
 'eval_precision': 0.8868186633457501,
 'eval_recall': 0.8865248226950354,
 'eval_f1': 0.8857048967267599,
 'eval_accuracy': 0.8865248226950354,
 'eval_runtime': 15.3671,
 'eval_samples_per_second': 27.526,
 'eval_steps_per_second': 3.449,
 'epoch': 10.0}

In [59]:
pt_save_directory = "./pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)


In [60]:

pt_model = AutoModelForSequenceClassification.from_pretrained("rpt_save_pretrained")

In [62]:
from transformers import pipeline

classifier = pipeline('text-classification', model="/content/transformer/pt_save_pretrained")
classifier("የቱርክ እግር ኳስ ፌዴሬሽን ሰኞ ዕለት አንድ የእግር ኳስ ክለብ ኃላፊ ዳኛ መመታቸውን ተከትሎ የከፍተኛው ሊግ እግር ኳስ ሙሉ በሙሉ እንዲቋረጥ አዟል። አንካራጉጁ የተባለው ክለብ ፕሬዝደንት ፋሩክ ኮቻ የመሐል ዳኛው ሐሊል መሙት ሜለርን በቡጢ መትተዋል። ፕሬዝደንቱ ይህን ድርጊት የፈፀሙት ቡድናቸው 1-0 እየመራ በ97ኛው ደቂቃ በተቀናቃኙ ክለብ ሪዜስፖር አንድ ጎል ከተቆጠረበት በኋላ ነው።")

[{'label': 'LABEL_3', 'score': 0.15869218111038208}]

# New Section